In [1]:
%reload_ext autoreload
%autoreload 2

## Experiment EXP-23-CA3091, EXP-24-CB4713, EXP-24-CB4719

In [2]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from pathlib import Path
%matplotlib qt5

In [3]:
all_data = []

In [4]:
DATA_DIR = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-23-CA3091/beadsLocalizationAccuracy/lowIntensityLocalizations/')

In [5]:
### Number of beads:
for i in range(1, 6):
    with open(DATA_DIR / Path('Pos' + str(i)), 'r') as fh:
        data = json.load(fh)
        print(f"Number of beads: {len(data)}")
        if len(data) != 0:
            all_data.extend(data)

Number of beads: 5
Number of beads: 7
Number of beads: 17
Number of beads: 32
Number of beads: 15


In [6]:
DATA_DIR = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4713/fluorChannelRegistration3D/localizations_vs_z/')
for filename in sorted(list(DATA_DIR.glob('*'))):
    with open(filename, 'r') as fh:
        data = json.load(fh)
    #print(f"Number of beads: {len(data)}")
        if len(data) != 0:
            all_data.extend(data)

DATA_DIR = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4719/fluorChannelRegistration3D/localizations_vs_z_101/')
for filename in sorted(list(DATA_DIR.glob('*'))):
    with open(filename, 'r') as fh:
        data = json.load(fh)
        #print(f"Number of beads: {len(data)}")
        if len(data) != 0:
            all_data.extend(data)

DATA_DIR = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4719/fluorChannelRegistration3D/localizations_vs_z_102/')
for filename in sorted(list(DATA_DIR.glob('*'))):
    with open(filename, 'r') as fh:
        data = json.load(fh)
        #print(f"Number of beads: {len(data)}")
        if len(data) != 0:
            all_data.extend(data)

DATA_DIR = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4719/fluorChannelRegistration3D/localizations_vs_z_103/')
for filename in sorted(list(DATA_DIR.glob('*'))):
    with open(filename, 'r') as fh:
        data = json.load(fh)
        #print(f"Number of beads: {len(data)}")
        if len(data) != 0:
            all_data.extend(data)

In [7]:
len(all_data)

1492

In [8]:
tuples = [(np.mean(one_bead['z']), np.std(one_bead['x']), np.std(one_bead['y']), np.std(one_bead['z']), len(one_bead['z'])) for one_bead in all_data
                     if len(one_bead['z']) > 30
         ]

In [9]:
counts = [len(one_bead['z']) for one_bead in all_data]

In [10]:
tuples

[(-324.3002199502516,
  13.721024084216538,
  19.219215107016442,
  36.91336343647204,
  191),
 (-174.59132787809898,
  11.136078063165888,
  16.41683397845811,
  24.44463806902528,
  199),
 (-371.353599887424,
  16.54861378352982,
  16.04293826743978,
  22.890384077774748,
  180),
 (-457.82861923121806,
  27.643114845940392,
  22.929584787902325,
  45.39937108542092,
  159),
 (-104.06136471542281,
  8.121376231826378,
  9.068334723370198,
  17.38148843952646,
  199),
 (-76.84095571799712,
  16.08183868204742,
  17.23500362496278,
  50.09987523299284,
  176),
 (-230.101774662099,
  12.580419797798434,
  17.39640250571133,
  38.76794381994023,
  188),
 (-460.08728770517934,
  11.224388787968312,
  19.5374012434504,
  31.304393699175066,
  193),
 (-102.98462070465088,
  7.102663347023492,
  9.938104112911567,
  21.37255198719308,
  200),
 (-383.3488358006333,
  13.159950944791452,
  11.06927026482664,
  19.48415371509459,
  198),
 (-483.07971217181233,
  23.953406287130264,
  26.28984024

In [11]:
z_values = [bead[0] for bead in tuples]
x_std = [bead[1] for bead in tuples]
y_std = [bead[2] for bead in tuples]
z_std = [bead[3] for bead in tuples]

plt.figure()
plt.plot(z_values, x_std, 'r*')
plt.show()

plt.figure()
plt.plot(z_values, y_std, 'b*')
plt.show()

plt.figure()
plt.plot(z_values, z_std, 'g*')
plt.show()

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
import pathlib
import pickle
from pathlib import Path
import matplotlib.animation as animation
import seaborn as sns
sns.set_style('white')
sys.setrecursionlimit(10000)
%matplotlib qt5

In [13]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.utils.plot_funcs import plot_od, plot_train_record
from cellbgnet.simulation.psf_kernel import SMAPSplineCoefficient
from cellbgnet.analyze_eval import spline_crlb_plot
from cellbgnet.utils.plot_funcs import plot_psf

In [14]:
model_path = Path('/mnt/sda1/SMLAT/training_runs/good_models/model_8865_venus.pkl')
with open(model_path, 'rb') as f:
    chromo_model = pickle.load(f)

In [15]:
calib_file = chromo_model.psf_params['calib_file']
crlb, rois = spline_crlb_plot(calib_file, z_range=600.0, photon_counts=2100.0, bg_photons=7.0, img_size=41)

Mean CRLB x over full z range:-600.0 - 600.0 [nm] is: 15.06832504272461
Mean CRLB y over full z range:-600.0 - 600.0 [nm] is  15.272810935974121
Mean CRLB z over full z range:-600.0 - 600.0 [nm] is  23.493881225585938


In [16]:
z = torch.arange(-600.0, 610.0, 10.0)

In [17]:
crlb.shape

torch.Size([121, 5])

In [18]:
z.shape

torch.Size([121])

In [22]:
plt.figure()
plt.plot(z, crlb[:, 0], 'b')
plt.plot(z_values, x_std, 'b*')
plt.xlim([-500, 500])
plt.xlabel('Z (nm)', fontsize=18)
plt.ylabel(r'$\sqrt{CRLB_{x}}$ (nm)', fontsize=18)
plt.show()

In [23]:
plt.figure()
plt.plot(z, crlb[:, 1], 'g')
plt.plot(z_values, y_std, 'g*')
plt.xlim([-500, 500])
plt.xlabel('Z (nm)', fontsize=18)
plt.ylabel(r'$\sqrt{CRLB_{y}}$ (nm)', fontsize=18)
plt.show()

In [24]:
plt.figure()
plt.plot(z, crlb[:, 2], 'r')
plt.plot(z_values, z_std, 'r*')
plt.xlim([-500, 500])
plt.xlabel('Z (nm)', fontsize=18)
plt.ylabel(r'$\sqrt{CRLB_{z}}$ (nm)', fontsize=18)
plt.show()